In [ ]:
!pip install transformers[torch]>=4.11.0
!pip install accelerate>=0.20.1

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
dataset=od.download('https://www.kaggle.com/datasets/bennijesus/lovecraft-fiction/data')

100%|██████████| 6.19M/6.19M [00:01<00:00, 4.87MB/s]


In [ ]:
pth='/content/lovecraft-fiction/data/concat.txt'

with open(pth, 'r', encoding='utf-8') as file:
    lyrics_content = file.read()
lines=lyrics_content.split('\n')

In [ ]:
len(lines)

46327

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(lines,test_size=0.15)
len(train),len(test)

(39377, 6950)

In [ ]:
with open('train.txt', 'w') as f:
  for t in train:
    f.write(t)
    f.write(' ')


with open('test.txt', 'w') as f:
  for t in test:
    f.write(t)
    f.write(' ')

train_path = 'train.txt'
test_path = 'test.txt'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import AutoModelWithLMHead, TrainingArguments

In [ ]:
from transformers import Trainer

training_args = TrainingArguments(
    output_dir="./gpt2-Taylor",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=64,
    eval_steps = 5,
    logging_steps=50,
    save_steps=0,
    warmup_steps=2,
    learning_rate = 2e-4
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
50,4.899400
100,4.700000
150,4.644800
200,4.617300


TrainOutput(global_step=243, training_loss=4.691315764752925, metrics={'train_runtime': 219.273, 'train_samples_per_second': 26.519, 'train_steps_per_second': 1.108, 'total_flos': 379853291520000.0, 'train_loss': 4.691315764752925, 'epoch': 1.0})

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub("story-generator")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kitty528/story-generator/commit/ef097beefb6812462cd67ae13ea35248968d09d5', commit_message='Upload model', commit_description='', oid='ef097beefb6812462cd67ae13ea35248968d09d5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

In [ ]:
generator = pipeline('text-generation', model='kitty528/story-generator', tokenizer='gpt2')

# Generate text

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: Once upon a time, I realized that that the light which I had seen was no shadow-cast out, nor an obstacle to escape. The great circle of red-faced men with the ghoulishly large hands, as the leader in the


In [ ]:

generated_text = generator("hi ", max_length=100, num_return_sequences=1)[0]['generated_text']

print("Generated Text:", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: hi  the old fainter with the grey, and he knew exactly how far his eyes must have been in Arkham, for I was soon to fall into despair. I’ll get to town soon, folks; then it will go away and come back again—though in no significant way. The whole story has been distorted and contrived by the ghouls. That is to say, the entire set of beings was a sort of gigantic insect thing with a single eye that could have penetrated
